# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [16]:
import requests
"""
    google REST api example
    each request needs to have a api key with the header named x-goog-api-key
    The request body needs to be in a json format
    Each request should have one Content object
    Each Content object is made up of Part objects
    A similar way to call this endpoint is to use the google genai library
    It does the request message body for you
"""


headers = {"x-goog-api-key": api_key, "Content-Type": "application/json"}

payload = {
    "contents":[
        {
            "role": "user",
            "parts": [
                {
                    "text": "Give me a fun fact"
                }
            ]
        }
    ]
}

payload

{'contents': [{'role': 'user', 'parts': [{'text': 'Give me a fun fact'}]}]}

In [17]:
response = requests.post(
    "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent",
    headers=headers,
    json=payload
)

response.json()

{'candidates': [{'content': {'parts': [{'text': "Here's a fun fact for you:\n\nA group of owls is called a **parliament**!"}],
    'role': 'model'},
   'finishReason': 'STOP',
   'index': 0}],
 'usageMetadata': {'promptTokenCount': 6,
  'candidatesTokenCount': 22,
  'totalTokenCount': 563,
  'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 6}],
  'thoughtsTokenCount': 535},
 'modelVersion': 'gemini-2.5-flash',
 'responseId': 'o4gSacLlCO2lqtsPiqn86Ac'}

In [18]:

print(response.json()["candidates"][0]["content"]["parts"][0]['text'])

Here's a fun fact for you:

A group of owls is called a **parliament**!


# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [10]:
# Create OpenAI client

from openai import OpenAI
geminiBaseUrl = "https://generativelanguage.googleapis.com/v1beta/openai/"
openapi = OpenAI(api_key = api_key, base_url=geminiBaseUrl)

messages = [{"role":"user","content":"tell me a fun fact"}]

response = openapi.chat.completions.create(model = "gemini-2.5-flash-preview-05-20", messages=messages)
print(response.choices[0].message.content)


Here's one for you:

A group of owls is called a **parliament**!


## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



In [ ]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-pro", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [ ]:
requests.get("http://localhost:11434").content

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
!ollama pull llama3.2

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [ ]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

In [ ]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

In [ ]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [4]:
#Not going to do it on a local model
#using gemini
from scraper import fetch_website_contents
from openai import OpenAI
geminiBaseUrl = "https://generativelanguage.googleapis.com/v1beta/openai/"
openapi = OpenAI(api_key = api_key, base_url=geminiBaseUrl)


#method to return website content
def getWebsiteContents(url:str):
    contents: str = fetch_website_contents(url)
    return contents

webSiteContents:str = getWebsiteContents("https://worldoftanks.com/")
webSiteContents

"\n    World of Tanks | Realistic Online Tank Game | Play for Free\n\n\nFailed to log in.\nLog in\nor\ncreate account\nMy Profile\nLog out\nWorld of Tanks\nMenu\nClose\nGame\nDownload Now\nMilitary Program\nNews\nUpdates\nRefer a Friend\nTankopedia\nMusic\nGuides\nNewcomer's Guide\nGeneral Guide\nGame Economics\nAccount Security\nAchievements\nFair Play Policy\nWargaming.net Game Center\nTwitch Drops Guide\nClans\nClan Portal\nMy Clan\nClan Ratings\nGlobal Map\nStronghold\nTournaments\nCommunity\nMy Profile\nSearch Players\nRatings\nDiscord\nMod Hub\nMedia\nCommunity Contributors\nCreate account\nCreate account\nONLINE TANK ACTION\nTake control of vehicles from World War\xa0II and the mid-20th century, competing against players around the world.\nCreate account\nABOUT THE GAME\nTo news\nPremium Shop\nLATEST NEWS\nRead all\nEqualize! Arcade Cabinet is Back!\nHonoring Heroes: Veterans Day Commemoration\nNiech żyje Polska! Let's Celebrate Polish Independence Day\nUse Huge Black Friday Dis

In [6]:
#develop messages list
messages = [{"role": "system","contents": "You are a avid gamer. Enthusiastically summarize the contents of the provided website. Ignore any promotional material or advertisement."},
            {"role": "user","content":f"{webSiteContents}"}]

model = "gemini-2.5-pro"
response = openapi.chat.completions.create(model=model, messages=messages)
print(response.choices[0].message.content)

Based on the text you provided, here is a detailed breakdown and summary of the World of Tanks website homepage.

### **Overall Summary**

This is the text from the homepage of **World of Tanks**, a popular free-to-play, online multiplayer game. The website serves as the central hub for both new and existing players, providing access to account management, game news, community features, and the in-game store. The user viewing this page is currently **logged out**.

---

### **Key Sections and Features**

The website is structured into several distinct sections:

**1. Account Management & Navigation:**
*   **Login/Registration:** Prominently features options to "Log in" or "create account." The message "Failed to log in" indicates a recent, unsuccessful login attempt.
*   **Main Menu:** Provides comprehensive navigation to all major areas of the game and community, including:
    *   **Game:** Download links, guides, news, and game information (Tankopedia, Music).
    *   **Clans:** Por